In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense, GRU
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import LeakyReLU
from keras.callbacks import ModelCheckpoint

from tqdm import tqdm

ModuleNotFoundError: No module named 'tensorflow'

In [13]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

X=[]
y=[]

folders = ["nadal", "djoko_sock", "federer", "alcaraz", "dimitrov_alcaraz", "dimitrov_thiem", "roland"]

for folder in folders:
    if not os.path.exists(f"tennis dataset antoine keller/{folder}/shots/"):
        print(f"dataset/{folder}/shots/ doesnt exist")
        continue
        
    print(f"Loading shots from dataset/{folder}/shots/")
        
    for shot_csv in tqdm(sorted(os.listdir(f'dataset/{folder}/shots/'))):
        data = pd.read_csv(os.path.join(f'dataset/{folder}/shots/', shot_csv))
        
        if folder == "nadal":
            revert_data= data.copy()
            for feature in data.columns:
                if feature[-2:]=="_x":
                    revert_data[feature] = 1 - data[feature]
            data = revert_data

        features = data.loc[:, data.columns != 'shot']

        X.append(features.to_numpy())
        y.append(data["shot"].iloc[0])
    
X = np.stack(X, axis=0)

y = np.array(y)
X = np.array(X)

print(f"Loaded {len(y)} shots for training")

X_test=[]
y_test=[]

folders = ["roland", "alcaraz"]
for folder in folders:
    for shot_csv in sorted(os.listdir(f'dataset/{folder}/shots/')):
    
        data = pd.read_csv(os.path.join(f'dataset/{folder}/shots/',shot_csv ))

        if folder == "dataset_nadal":
                revert_data= data.copy()
                for feature in data.columns:
                    if feature[-2:]=="_x":
                        revert_data[feature] = 1 - data[feature]
                data = revert_data


        features = data.loc[:, data.columns != 'shot']

        X_test.append(features.to_numpy())
        y_test.append(data["shot"].iloc[0])
    
X_test = np.stack(X_test, axis=0)

y_test = np.array(y_test)
X_test = np.array(X_test)

print(f"Loaded {len(y_test)} shots for test")

Loading shots from dataset/nadal/shots/


100%|███████████████████████████████████████████████████████████████████████████████| 442/442 [00:03<00:00, 140.98it/s]


Loading shots from dataset/djoko_sock/shots/


100%|███████████████████████████████████████████████████████████████████████████████| 247/247 [00:00<00:00, 406.56it/s]


Loading shots from dataset/federer/shots/


...

dataset/dimitrov_alcaraz/shots/ doesnt exist
Loading shots from dataset/dimitrov_thiem/shots/


100%|███████████████████████████████████████████████████████████████████████████████| 143/143 [00:00<00:00, 411.13it/s]


Loading shots from dataset/roland/shots/


100%|█████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 345.54it/s]


Loaded 1350 shots for training
Loaded 193 shots for test


In [16]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, shuffle= True)


print(f"Shape of train features : {X_train[0].shape}")
print(f"Shape of val features : {X_val[0].shape}")

print("Total categories: ", len(np.unique(y_train)))
print("Total categories: ", len(np.unique(y_val)))

nb_cat = len(np.unique(y_train))

Shape of train features : (30, 26)
Shape of val features : (30, 26)
Total categories:  4
Total categories:  4


In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=nb_cat)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=nb_cat)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=nb_cat)

y_train = np.array(y_train)
X_train = np.array(X_train)
#X_train = np.expand_dims(X_train, axis=-1)

y_val = np.array(y_val)
X_val = np.array(X_val)
#X_val = np.expand_dims(X_val, axis=-1)

y_test = np.array(y_test)
X_test = np.array(X_test)

In [18]:
assert len(le.classes_) == nb_cat

In [19]:
print("X_train Shape: ", X_train.shape)
print("X_val Shape: ", X_val.shape)
print("X_test Shape: ", X_test.shape)
print("y_train Shape: ", y_train.shape)
print("y_val Shape: ", y_val.shape)
print("y_test Shape: ", y_test.shape)

X_train Shape:  (904, 30, 26)
X_val Shape:  (446, 30, 26)
X_test Shape:  (193, 30, 26)
y_train Shape:  (904, 4)
y_val Shape:  (446, 4)
y_test Shape:  (193, 4)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()

# First LSTM layer (returns sequences for stacking)
model.add(LSTM(units=32, return_sequences=True, input_shape=(30, 26)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(units=24))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Dense layer
model.add(Dense(units=16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=nb_cat, activation='softmax'))

# Output layer
model.add(Dense(units=nb_cat, activation='softmax'))

m1.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
m1.summary()

hist = m1.fit(X_train, y_train,
               validation_data=(X_val, y_val),
               batch_size = 32,
                epochs=300, 
                verbose = 1
             )

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 24)                  │           3,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8)                   │             200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │              36 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,980 (15.55 KB)

 Trainable params: 3,980 (15.55 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9756 - loss: 0.0971 - val_accuracy: 0.9619 - val_loss: 0.1868
Epoch 2/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9713 - loss: 0.0914 - val_accuracy: 0.9529 - val_loss: 0.2073
Epoch 3/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9666 - loss: 0.1157 - val_accuracy: 0.9417 - val_loss: 0.2285
Epoch 4/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9765 - loss: 0.0615 - val_accuracy: 0.9529 - val_loss: 0.1968
Epoch 5/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9630 - loss: 0.0788 - val_accuracy: 0.9507 - val_loss: 0.2310
Epoch 6/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9830 - loss: 0.0554 - val_accuracy: 0.9507 - val_loss: 0.2319
Epoch 7/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9824 - loss: 0.0669 - val_accuracy: 0.9552 - val_loss: 0.2096
Epoch 8/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9724 - loss: 0.0835 - val_accuracy: 0.

In [ ]:
model.save('large_model.h5')